In [1]:
import yfinance as yf
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import pandas as pd
import plotly.graph_objects as go

In [2]:
ticker = 'icbp.jk'

In [3]:
ticks = yf.Ticker(ticker)

In [4]:
# 1d,5d,1mo,-6mo, 1y, 5y, 10y, ytd, max
data = ticks.history(period='5y').reset_index()
data.tail()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
1247,2023-01-19 00:00:00+07:00,10475.0,10500.0,10375.0,10500.0,3508100,0.0,0.0
1248,2023-01-20 00:00:00+07:00,10500.0,10525.0,10325.0,10500.0,2174100,0.0,0.0
1249,2023-01-24 00:00:00+07:00,10475.0,10475.0,10275.0,10400.0,1978500,0.0,0.0
1250,2023-01-25 00:00:00+07:00,10400.0,10450.0,10300.0,10425.0,1422900,0.0,0.0
1251,2023-01-26 00:00:00+07:00,10425.0,10425.0,10000.0,10075.0,11246000,0.0,0.0


In [5]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])
fig.update_layout(title=f'Historical Data of {ticker.upper()} Stock')
fig.show()

In [6]:
df = data[['Date','Close']].rename(columns={'Date':'ds','Close':'y'})
df.head()

,ds,y
0,2018-01-26 00:00:00+07:00,7735.606934
1,2018-01-29 00:00:00+07:00,7779.435547
2,2018-01-30 00:00:00+07:00,7626.039062
3,2018-01-31 00:00:00+07:00,7647.951660
4,2018-02-01 00:00:00+07:00,7626.039062


In [7]:
df['ds'] = df['ds'].dt.tz_localize(None)
df.tail()

,ds,y
1247,2023-01-19,10500.0
1248,2023-01-20,10500.0
1249,2023-01-24,10400.0
1250,2023-01-25,10425.0
1251,2023-01-26,10075.0


In [8]:
m = Prophet()
m.fit(df)

21:37:07 - cmdstanpy - INFO - Chain [1] start processing
21:37:09 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
future = m.make_future_dataframe(periods=60)
future.tail()

,ds
1307,2023-03-23
1308,2023-03-24
1309,2023-03-25
1310,2023-03-26
1311,2023-03-27


In [11]:
forecast = m.predict(future)
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
1307,2023-03-23,9906.890063,8957.704869,9928.830659,9865.353727,9941.696238,-471.935762,-471.935762,-471.935762,10.884214,10.884214,10.884214,-482.819976,-482.819976,-482.819976,0.0,0.0,0.0,9434.954301
1308,2023-03-24,9911.551580,8921.444635,9865.191182,9868.009462,9947.762505,-499.361733,-499.361733,-499.361733,4.108898,4.108898,4.108898,-503.470631,-503.470631,-503.470631,0.0,0.0,0.0,9412.189847
1309,2023-03-25,9916.213097,8918.944412,9822.892793,9871.813450,9952.885983,-536.484363,-536.484363,-536.484363,-13.344750,-13.344750,-13.344750,-523.139613,-523.139613,-523.139613,0.0,0.0,0.0,9379.728734
1310,2023-03-26,9920.874614,8907.372824,9866.925912,9875.699051,9958.633571,-555.070875,-555.070875,-555.070875,-13.344752,-13.344752,-13.344752,-541.726123,-541.726123,-541.726123,0.0,0.0,0.0,9365.803738
1311,2023-03-27,9925.536131,8869.788873,9839.410847,9881.243656,9964.499913,-561.078367,-561.078367,-561.078367,-1.941336,-1.941336,-1.941336,-559.137032,-559.137032,-559.137032,0.0,0.0,0.0,9364.457763


In [12]:
forecast[['ds','yhat','yhat_lower','yhat_upper']]

,ds,yhat,yhat_lower,yhat_upper
0,2018-01-26,8310.440491,7823.846933,8765.273886
1,2018-01-29,8268.207821,7765.788814,8755.438435
2,2018-01-30,8264.991768,7798.046018,8717.675637
3,2018-01-31,8246.435963,7794.341808,8720.509714
4,2018-02-01,8237.119868,7786.748050,8709.786575
...,...,...,...,...
1307,2023-03-23,9434.954301,8957.704869,9928.830659
1308,2023-03-24,9412.189847,8921.444635,9865.191182
1309,2023-03-25,9379.728734,8918.944412,9822.892793
1310,2023-03-26,9365.803738,8907.372824,9866.925912


In [13]:
plot_plotly(m,forecast)

In [14]:
plot_components_plotly(m,forecast)